In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# df = pd.read_csv("./results/mdi_local.real_data_regression/diabetes_regression/varying_sample_row_n/seed331/results.csv")
# df = pd.read_csv("./results/mdi_local.real_data_classification/diabetes_classification/varying_sample_row_n/seed331/results.csv")
# df = pd.read_csv("./results/mdi_local.real_data_classification/fico/varying_sample_row_n/seed331/results.csv")
# df = pd.read_csv("./results/mdi_local.real_data_regression/satellite_image/varying_sample_row_n/seed331/results.csv")
# df = pd.read_csv("./results/mdi_local.real_data_classification/juvenile/varying_sample_row_n/seed331/results.csv")
# df = pd.read_csv("./results/mdi_local.real_data_regression/ca_housing/varying_sample_row_n/seed331/results.csv")

In [ ]:
#################### Change the following according to the dataset ####################
task = "classification" #"regression" "classification"
########################################################################################
methods = ["LFI_with_raw","LFI_without_raw","TreeSHAP", "LIME"]
nreps = df["rep"].max()
n_testsize = df[['train_size', 'test_size']].drop_duplicates()
num_features = df['num_features'].drop_duplicates()[0]
metrics = {"regression": "MSE", "classification": "AUC"}
results = {}

for m in methods:
    results[m] = {}
    for r in range(nreps+1):
        results[m][r] = []

In [ ]:
for m in methods:
    for r in range(nreps+1):
        df_sub = df[(df["fi"] == m) & (df["rep"] == r)]
        assert len(df_sub[metrics[task]+"_before_ablation"].unique()) == 1
        results[m][r].append(df_sub[metrics[task]+"_before_ablation"].unique()[0])
        for i in range(num_features):
            assert len(df_sub[metrics[task]+f"_after_ablation_{i+1}"].unique()) == 1
            results[m][r].append(df_sub[metrics[task]+f"_after_ablation_{i+1}"].unique()[0])

In [ ]:
agg_results = {} 
for m in methods:
    agg_results[m] = None

In [ ]:
for m in methods:
    result = []
    for f in range(num_features+1):
        result.append(np.mean([results[m][r][f] for r in range(nreps+1)]))          
    agg_results[m] = result

In [ ]:
# Plotting
fig, ax = plt.subplots()
for m in methods:
    if m in ["TreeSHAP", "LIME"]:
        ax.plot(range(num_features+1), agg_results[m], label=m, linestyle='dashed')
    else:
        ax.plot(range(num_features+1), agg_results[m], label=m)
ax.set(xlabel='Number of features ablated', ylabel=metrics[task],
        title=f'Train size = {n_testsize["train_size"].values[0]}, Test size = {n_testsize["test_size"].values[0]}')
ax.legend()
plt.savefig(f"ablation_fico.png")
plt.show()